# Predictive Modeling – Gas Corrosion Risk

In [ ]:
import pandas as pd, os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
assets = pd.read_csv('asset_registry.csv'); labels = pd.read_csv('labels_gas_corrosion.csv')
X_cols=['Region','System','AssetType','Material','CoatingType','SeamType','ClassLocation','HCA_Flag','SoilCorrosivityIndex','MAOP_psig','PipeToSoilPotential_mV','AgeYears','DiameterInches','CPStatus','CorrosionRiskScore_Rubric']
df = assets[assets['Commodity']=='Gas'][['AssetID']+X_cols].merge(labels,on='AssetID'); X=df[X_cols]; y=df['CorrosionFinding']
pre=ColumnTransformer([('num',Pipeline([('imp',SimpleImputer(strategy='median'))]),['SoilCorrosivityIndex','MAOP_psig','PipeToSoilPotential_mV','AgeYears','DiameterInches','CorrosionRiskScore_Rubric','ClassLocation']),('cat',Pipeline([('imp',SimpleImputer(strategy='most_frequent')),('oh',OneHotEncoder(handle_unknown='ignore'))]),['Region','System','AssetType','Material','CoatingType','SeamType','HCA_Flag','CPStatus'])])
Xtr,Xte,ytr,yte=train_test_split(X,y,test_size=0.25,random_state=42,stratify=y)
logit=Pipeline([('pre',pre),('clf',LogisticRegression(max_iter=400))]).fit(Xtr,ytr)
print('AUC:',roc_auc_score(yte,logit.predict_proba(Xte)[:,1]))